# **Init**

In [1]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import random
import numpy as np

print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

True


# **Formatting of training data into pixel values to be inputted into the Neural Network**

In [2]:
data = []

def create_training_data():
  file_path = os.getcwd() + r"\content\seg_train"
  types = ['buildings','forest','glacier','mountain','sea','street']
  for category in types:
    class_num = types.index(category)
    path = os.path.join(file_path,category)
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path,img), 0)
      if img_array.shape == (150,150):
        img_array = np.reshape(img_array,22500)
        data.append([img_array,class_num])

create_training_data()
data = np.asarray(data)
random.shuffle(data)
x = data[:,0]
y = torch.zeros(13986,6)
x = x.tolist()
Y = data[:,1]
Y = Y.tolist()
y = y.cuda()
y.to(device)
x = torch.cuda.FloatTensor(x)
x = x/255
print(x.is_cuda)
y_data = torch.cuda.FloatTensor(Y)

for n in range(13986):
  if y_data[n] == 5:
    y[n,5] += 1
  elif y_data[n] == 4:
    y[n,4] += 1
  elif y_data[n] == 3:
    y[n,3] += 1
  elif y_data[n] == 2:
    y[n,2] += 1
  elif y_data[n] == 1:
    y[n,1] += 1
  elif y_data[n] == 0:
    y[n,0] += 1

print(y.is_cuda)

C:\Users\setho\AppData\Local\Temp\ipykernel_44932\1486572466.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray(data)
C:\Users\setho\AppData\Local\Temp\ipykernel_44932\1486572466.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:210.)
  x = torch.cuda.FloatTensor(x)


True
True


# **Creation of Convolutional Neural Network Model and Training the model using Training Data**

In [ ]:

class NeuralNetwork(nn.Module):
  def __init__(self,n_input,n_hidden_1,n_hidden_2,n_output):
    super(NeuralNetwork,self).__init__()
    self.l1 = nn.Linear(n_input,n_hidden_1)
    self.softmax = nn.Softmax()
    self.l2 = nn.Linear(n_hidden_1,n_hidden_2)
    self.l3 = nn.Linear(n_hidden_2,n_output)

  def forward(self,X):
    out = self.l1(X)
    out = self.softmax(out)
    out = self.l2(out)
    out = self.softmax(out)
    out = self.l3(out)
    out = self.softmax(out)
    return out

input_features = 22500
hidden_feature_1 = 128
hidden_feature_2 = 128
output_features = 6
model = NeuralNetwork(input_features,hidden_feature_1,hidden_feature_2,output_features)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.005
optimiser = torch.optim.Adam(model.parameters(),lr=learning_rate)
iterations = 1000000
model.to(device).eval()

for epoch in range(iterations):
  y_predicted = model(x)
  loss = criterion(y_predicted,y)
  loss.backward()
  optimiser.step()
  print('epoch: ' , epoch+1)
  if epoch+1 == iterations:
    print('Training Complete')

torch.save(model, os.getcwd() + r'\content\model\model5.pth')

# **Testing of Convolutional Neural Network in Testing Sequence using Testing Data**

In [ ]:

torch.load(os.getcwd() + r'\content\model\model4.pth')


data_test = []
data_color = []
test_set = int(input('Test set number: '))

def testing_data():
  file_path = os.getcwd() + r'\content\seg_test'
  types = ['buildings','forest','glacier','mountain','sea','street']
  for category in types:
    class_num = types.index(category)
    path = os.path.join(file_path,category)  
    for img in os.listdir(path):
      img_array_test = cv2.imread(os.path.join(path,img), 0)
      img_array_color = cv2.imread(os.path.join(path,img), 1)
      
      if img_array_test.shape == (150,150):
        img_array_test_reshape = np.reshape(img_array_test,22500)
        img_array_color_reshape = np.reshape(img_array_color,(150,150,3))
        data_test.append([img_array_test_reshape,class_num])
        data_color.append(img_array_color)

testing_data()
data_test = np.asarray(data_test)
plt.imshow(data_color[test_set])
x_test = data_test[:,0]
y_test = torch.zeros(1000,6)
x_test = x_test.tolist()
Y_test = data_test[:,1]
Y_test = Y_test.tolist()
x_test = torch.cuda.FloatTensor(x_test)
x_test = x_test/255
y_test_data = torch.cuda.FloatTensor(Y_test)


def test(x_test,Y_test):
  if model(x_test[test_set])[0] > 0.6:
    print('Predicted feature: buildings')
  elif model(x_test[test_set])[1] > 0.6:
    print('Predicted feature: forest')
  elif model(x_test[test_set])[2] > 0.6:
    print('Predicted feature: glacier')
  elif model(x_test[test_set])[3] > 0.6:
    print('Predicted feature: mountain')
  elif model(x_test[test_set])[4] > 0.6:
    print('Predicted feature: sea')
  elif model(x_test[test_set])[5] > 0.6:
    print('Predicted feature: street')
  else:
    print('No Prediction')

  if Y_test[test_set] == 0:
    print('Actual feature: buildings')
  elif Y_test[test_set] == 1:
    print('Actual feature: forest')
  elif Y_test[test_set] == 2:
    print('Actual feaure: glacier')
  elif Y_test[test_set] == 3:
    print('Actual feature: mountain')
  elif Y_test[test_set] == 4:
    print('Actual feature: sea')
  elif Y_test[test_set] == 5:
    print('Actual feature: street')

test(x_test,Y_test)